# Лабораторная работа 5. Вариант 3. Группа 343

## Задание
Найти решение задачи

$\frac{\partial u}{\partial t}=Lu+f(x,t),0<x<1,0<1\leq0.1,$

$u(x,0)=\varphi(x),0\leq x\leq1$

$(\alpha_1(t)u-\alpha_2(t)\frac{\partial u}{\partial x})|_{x=0}=\alpha(t), 0\leq t\leq0.1,$

$(\beta_1(t)u+\beta_2(t)\frac{\partial u}{\partial x})|_{x=1}=\beta(t), 0\leq t\leq0.1,$

используя различные разностные схемы:

+ явную схему порядка $O(h^2+\tau)$ с аппроксимацией производных в граничных условиях с порядком $O(h^2)$;
+ схему с весами при $\sigma=0,\sigma=1,\sigma=1/2$ с аппроксимацией производных в граничных условиях с порядком $O(h)$.

### Условие
$\frac{\partial u}{\partial t}=\frac{\partial^2 u}{\partial {x^2}}+\frac{\partial u}{\partial x}+f(x,t),$

$u(x,0)=\varphi(x),0\leq x\leq 1,$

$u(0,t)=\alpha(t), \frac{\partial u}{\partial x}|_{x=1}=\beta(t),0\leq t\leq 0.1.$

### Функции для тестирования
$u_1(x, t) = x^3 + t^3$

$u_2(x, t) = x^3t^3$

$u_3(x, t) = cos(2x) + sin(2t + 1)$

$u_4(x, t) = cos(2x)sin(2t + 1)$

$u_5(x, t) = 2x - t$

In [1]:
import numpy as np
from numpy import linalg as LA
from tabulate import tabulate

N = 5
M = 5
X_RANGE = (0, 1)
T_RANGE = (0, 0.1)

ALPHA_1 = 1
ALPHA_2 = 0
BETA_1 = 0
BETA_2 = 1

In [2]:
# (function, first derivative with respect to x, second derivative with respect to x, first derivative with respect to t)
def u1(x, t):
    u = x ** 3 + t ** 3
    dxu = 3 * x ** 2
    d2xu = 6 * x
    dtu = 3 * t ** 2
    return (u, dxu, d2xu, dtu)


def u2(x, t):
    u = x ** 3 * t ** 3
    dxu = 3 * x ** 2 * t ** 3
    d2xu = 6 * x * t ** 3
    dtu = 3 * x ** 3 * t ** 2
    return (u, dxu, d2xu, dtu)


def u3(x, t):
    u = np.cos(2 * x) + np.sin(2 * t + 1)
    dxu = -2 * np.sin(2 * x)
    d2xu = -4 * np.cos(2 * x)
    dtu = 2 * np.sin(2 * t + 1)
    return (u, dxu, d2xu, dtu)


def u4(x, t):
    u = np.cos(2 * x) * np.sin(2 * t + 1)
    dxu = -2 * np.sin(2 * x) * np.sin(2 * t + 1)
    d2xu = -4 * np.cos(2 * x) * np.sin(2 * t + 1)
    dtu = 2 * np.cos(2 * t + 1) * np.cos(2 * x)
    return (u, dxu, d2xu, dtu)

def u5(x, t):
    u = 2 * x - t
    dxu = 2
    d2xu = 0
    dtu = -1
    return (u, dxu, d2xu, dtu)


US = [u1, u2, u3, u4, u5]

In [3]:
def a(x, t):
    return 1


def b(x, t):
    return 1


def c(x, t):
    return 0

In [4]:
def find_system_solution_gauss(A, b, use_pivoting=False):
    n = len(A)
    for i in range(n - 1):
        if use_pivoting:
            pivot_index = abs(A[i:, i]).argmax() + i
            if pivot_index != i:
                A[[i, pivot_index]] = A[[pivot_index, i]]
                b[[i, pivot_index]] = b[[pivot_index, i]]
                
        for j in range(i + 1, n):
            ratio = A[j, i] / A[i, i]
            A[j, i:] -= ratio * A[i, i:]
            b[j] -= ratio * b[i]

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]

    return x

In [5]:
def find_tables_norm(table_a, table_b):
    res = -1
    for i in range(0, 5):
        for j in range(0, 5):
            a = np.abs(table_a[i, j])
            b = np.abs(table_b[i, j])
            res = np.max(res, np.abs(a - b))
    return res


def find_accurate_solution(u, x, t):
    solution = u(x, t)[0]
    f_value = u(x, t)[3] - u(x, t)[2] - u(x, t)[1]
    return (solution, f_value)


def find_accurate_solutions(n, m, u):
    res = []
    for k in range(0, m + 1):
        res.append([])
        for i in range(1, n + 1):
            val = find_accurate_solution(u, i * X_RANGE[1] / n, k * T_RANGE[1] / m)[0]
            res[k].append(val)
            
    return res

$u_0^k(\alpha_1(t_k)+\frac{3}{2h}\alpha_2(t_k))-\alpha_2(t_k)\frac{4u_1^k-u_2^k}{2h}=\alpha(t_k)$

$u_0^k=\frac{\alpha_2(t_k)\frac{4u_1^k-u_2^k}{2h}+\alpha(t_k)}{\alpha_1(t_k)+\frac{3}{2h}\alpha_2(t_k)}$

$u_0^k=\frac{\alpha_2(t_k)(4u_1^k-u_2^k)+2h\alpha(t_k)}{2h\alpha_1(t_k)+3\alpha_2(t_k)}$


---


$u_N^k(\beta_1(t_k)+\frac{3}{2h}\beta_2(t_k))-\beta_2(t_k)\frac{4u_{N-1}^k - u_{N-2}^k}{2h}=\beta(t_k)$

$u_N^k=\frac{\beta_2(t_k)\frac{4u_{N-1}^k - u_{N-2}^k}{2h}+\beta(t_k)}
{\beta_1(t_k)+\frac{3}{2h}\beta_2(t_k)}$

$u_N^k=\frac{\beta_2(t_k)(4u_{N-1}^k - u_{N-2}^k)+2h\beta(t_k)}{2h\beta_1(t_k)+3\beta_2(t_k)}$


---


$L_hu_i^k=a(x_i, t_k)\frac{u_{i+1}^k-2u_i^k+u_{i-1}^k}{h^2}+b(x_i,t_k)\frac{u_{i+1}^k-u_{i-1}^k}{2h}+c(x_i, t_k)u_i^k$

In [6]:
def find_solution_grid(n, m, u, a, b, c):
    step_x = (X_RANGE[1] - X_RANGE[0]) / n
    step_t = (T_RANGE[1] - T_RANGE[0]) / m
    # us[i, k] <-> (u_i)^k
    us = np.zeros((n + 1, m + 1))
    # u: (x, t) -> (u, dxu, d2xu, dtu)
    def put_u_i_k(i, k):
        x_i = X_RANGE[0] + i * step_x
        t_k1 = T_RANGE[0] + (k - 1) * step_t
        t_k = t_k1 + step_t
        
        if k == 0:
            us[i, k] = u(x_i, T_RANGE[0])[0]
            return
        
        # find (u_0)^k
        if i == 0:
            alpha = ALPHA_1 * u(0, t_k)[0] + ALPHA_2 * u(0, t_k)[1]
            u0k =  ALPHA_2 * (4 * us[1, k] - us[2, k]) + 2 * step_x * alpha
            u0k /= 2 * step_x * ALPHA_1 + 3 * ALPHA_2
            us[i, k] = u0k
            return
        
        #find (u_N)^k
        if i == n:
            beta = BETA_1 * u(1, t_k)[0] + BETA_2 * u(1, t_k)[1]
            uNk = BETA_2 * (4 * us[n - 1, k] - us[n - 2, k]) + 2 * step_x * beta
            uNk /=2 * step_x * BETA_1 + 3 * BETA_2
            us[i, k] = uNk
            return
    
        # (u_i)^k-1
        u_prev = us[i, k - 1]
        # find (u_i)^k
        res = u_prev
        
        function_val = find_accurate_solution(u, x_i, t_k1)[1]
        # find Lh
        lh = \
            a(x_i, t_k) * (us[i + 1, k] - 2 * us[i, k] + us[i - 1, k]) / step_x ** 2 + \
            0.5 * b(x_i, t_k) * (us[i + 1, k] - us[i - 1, k]) / step_x + \
            c(x_i, t_k) * us[i, k]
        
        us[i, k] = u_prev + step_t * (lh * u_prev + function_val)
        
    for i in range(0, n + 1):
        put_u_i_k(i, 0)
        
    for k in range(1, m + 1):
        for i in range(1, n):
            put_u_i_k(i, k)
        put_u_i_k(0, k)
        put_u_i_k(n, k)
            
    return us

In [7]:
def find_solution_grid_weights(sigma, n, m, u, a, b, c):
    step_x = (X_RANGE[1] - X_RANGE[0]) / n
    step_t = (T_RANGE[1] - T_RANGE[0]) / m
    # us[i, k] <-> (u_i)^k
    us = np.zeros((n + 1, m + 1))
    def put_u_i_k(i, k):
        # find (u_i)^0
        x_i = X_RANGE[0] + i * step_x
        t_k1 = T_RANGE[0] + (k - 1) * step_t
        t_k = t_k1 + step_t
        
        if k == 0:
            us[i, k] = u(x_i, T_RANGE[0])[0]
            return
        
        pass
        
    for i in range(0, n + 1):
        put_u_i_k(i, 0)
    
    return us

In [8]:
def print_result(u, is_explicit=True, sigma=0):
    if is_explicit:
        print('Явный результат:')
        us = find_solution_grid(N, M, u, a, b, c)
    else:
        print(f'Неявный результат для веса {sigma}:')
        us = find_solution_grid_weights(sigma, N, M, u, a, b, c)
        
    us_headers = ['t \ x']
    for i in range(0, N + 1):
        us_headers.append(str(round(X_RANGE[0] + i * (X_RANGE[1] - X_RANGE[0]) / N, 2)))
        
    us_table = []
    for k in range(0, M + 1):
        us_table.append([str(round(T_RANGE[0] + k * (T_RANGE[1] - T_RANGE[0]) / M, 2))])
        
    for i in range(0, N + 1):
        for k in range(0, M + 1):
            us_table[k].append(str(us[i, k]))            
        
    print(tabulate(us_table, headers=us_headers))
    
    print('Точность:')
    if is_explicit:
        acc_headers =  ['h', 'tau', '||J_ex - u^(h, tau)||', '||u^(h, tau) - u^(2h, tau1)||']
    else:
        acc_headers =  ['h', 'tau', '||J_ex - u^(h, tau)||', '||u^(h, tau) - u^(2h, tau)||']
        
    acc_table = []
    
    for div in [10, 20, 40]:
        n = div
        m = div
        row = []
        row.append((T_RANGE[1] - T_RANGE[0]) / M)
        accurate_table = find_accurate_solutions(n, m, u)
        if is_explicit:
            table_a = find_solution_grid(n, m, u, a, b, c)
            table_b = find_solution_grid(n // 2, m // 2, u, a, b, c)
        else:
            table_a = find_solution_grid_weights(sigma, n, m, u, a, b, c)
            table_b = find_solution_grid_weights(sigma, n // 2, m // 2, u, a, b, c)
        
        acc_table.append([])
    
    print(tabulate(acc_table, headers=acc_headers))

In [9]:
for i in range(0, len(US)):
    print(f'Результат для u_{i + 1}\n')
    print_result(US[i], True)
    print()
    print_result(US[i], False, 0)
    print_result(US[i], False, 1)
    print_result(US[i], False, 0.5)
    print('\n\n\n\n')

Результат для u_1

Явный результат:
  t \ x       0.0        0.2          0.4         0.6          0.8       1.0
-------  --------  ---------  -----------  ----------  -----------  --------
   0     0          0.008      0.064        0.216       0.512       1
   0.02  8e-06     -0.0184     0.00587008   0.122971    0.405932    0.900253
   0.04  6.4e-05   -0.044776  -0.0518242    0.0265268   0.276402    0.759694
   0.06  0.000216  -0.07108   -0.107671    -0.0682625   0.133608    0.600898
   0.08  0.000512  -0.097264  -0.160342    -0.156721   -0.00999909  0.438908
   0.1   0.001     -0.12328   -0.208663    -0.235221   -0.142957    0.287798
Точность:
h    tau    ||J_ex - u^(h, tau)||    ||u^(h, tau) - u^(2h, tau1)||
---  -----  -----------------------  -------------------------------

Неявный результат для веса 0:
  t \ x    0.0    0.2    0.4    0.6    0.8    1.0
-------  -----  -----  -----  -----  -----  -----
   0         0  0.008  0.064  0.216  0.512      1
   0.02      0  0      0    

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in double_scalars
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in double_scalars


h    tau    ||J_ex - u^(h, tau)||    ||u^(h, tau) - u^(2h, tau)||
---  -----  -----------------------  ------------------------------





Результат для u_2

Явный результат:
  t \ x    0.0          0.2           0.4          0.6          0.8          1.0
-------  -----  -----------  ------------  -----------  -----------  -----------
   0         0   0            0            0            0            0
   0.02      0   0            0            0            0            3.2e-06
   0.04      0  -1.92e-08     1.0752e-06   4.4352e-06   1.12128e-05  3.9072e-05
   0.06      0  -9.408e-07    3.5328e-06   1.91808e-05  5.17633e-05  0.000149024
   0.08      0  -4.9152e-06   4.91519e-06  4.56192e-05  0.000133326  0.000367362
   0.1       0  -1.536e-05   -4.2243e-11   8.064e-05    0.000261126  0.000721288
Точность:
h    tau    ||J_ex - u^(h, tau)||    ||u^(h, tau) - u^(2h, tau1)||
---  -----  -----------------------  -------------------------------

Неявный результат для веса 0:
  t \ x    0.0 

h    tau    ||J_ex - u^(h, tau)||    ||u^(h, tau) - u^(2h, tau)||
---  -----  -----------------------  ------------------------------





Результат для u_5

Явный результат:
  t \ x    0.0    0.2       0.4      0.6        0.8        1.0
-------  -----  -----  --------  -------  ---------  ---------
   0      0      0.4   0.8       1.2        1.6        2
   0.02  -0.02   0.34  0.8624    1.6057     2.6961     3.32624
   0.04  -0.04   0.28  0.911062  2.204      5.31009    6.61212
   0.06  -0.06   0.22  0.941258  3.07753   12.604     16.0461
   0.08  -0.08   0.16  0.949028  4.33183   37.1132    48.307
   0.1   -0.1    0.1   0.931734  6.08809  138.73     183.211
Точность:
h    tau    ||J_ex - u^(h, tau)||    ||u^(h, tau) - u^(2h, tau1)||
---  -----  -----------------------  -------------------------------

Неявный результат для веса 0:
  t \ x    0.0    0.2    0.4    0.6    0.8    1.0
-------  -----  -----  -----  -----  -----  -----
   0         0    0.4    0.8    1.2    1.6      2
   0.